In [6]:

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# Complete code with all necessary imports for processing the images and visualizing optical flow


# Function to draw optical flow on images
def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    # create line endpoints
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    # create image and draw
    vis = np.copy(img)
    vis = np.dstack((vis, vis, vis))  # convert to 3 channels
    for (x1, y1), (x2, y2) in lines:
        cv2.line(vis, (x1, y1), (x2, y2), (0, 255, 0), 1)
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

# # Load the consecutive images
# image_files = ['SGA2101411S0014\SGA2101411S0014IMG000_1.jpg', 'SGA2101411S0014\SGA2101411S0014IMG000_2.jpg',
#                'SGA2101411S0014\SGA2101411S0014IMG000_3.jpg', 'SGA2101411S0014\SGA2101411S0014IMG000_4.jpg',
#                'SGA2101411S0014\SGA2101411S0014IMG000_5.jpg', 'SGA2101411S0014\SGA2101411S0014IMG000_6.jpg',
#                'SGA2101411S0014\SGA2101411S0014IMG000_7.jpg', 'SGA2101411S0014\SGA2101411S0014IMG000_8.jpg']
# Load the consecutive images
image_files = ['Youtube_assault\YT_1.png','Youtube_assault\YT_2.png',
               'Youtube_assault\YT_3.png','Youtube_assault\YT_4.png']
# Read and convert the images to grayscale, ensure they are the same size
gray_images = []
for file in image_files:
    image = cv2.imread(file)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_images.append(gray)

# Make sure all images are the same size
if any(gray.shape != gray_images[0].shape for gray in gray_images):
    # If not, resize images to match the first image's size
    reference_shape = gray_images[0].shape
    gray_images = [cv2.resize(gray, (reference_shape[1], reference_shape[0])) for gray in gray_images]

# # Read and convert the images to grayscale
# gray_images = [cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2GRAY) for file in image_files]

# Calculate optical flow between each pair of consecutive images
flow_list = []
for i in range(len(gray_images)-1):
    flow = cv2.calcOpticalFlowFarneback(gray_images[i], gray_images[i+1], None, 0.5, 3, 15, 3, 5, 1.2, 0)
    flow_list.append(flow)

# Visualize the optical flow on the grayscale images
optical_flow_images = [draw_flow(gray_images[i], flow_list[i]) for i in range(len(flow_list))]

# Save the optical flow images to files and return the file paths
optical_flow_image_paths = []
for i, of_image in enumerate(optical_flow_images):
    path = f'Youtube_assault\YT_optical_flow_image_{i}.png'
    cv2.imwrite(path, of_image)
    optical_flow_image_paths.append(path)

# Provide the paths to the saved optical flow images
optical_flow_image_paths


['Youtube_assault\\YT_optical_flow_image_0.png',
 'Youtube_assault\\YT_optical_flow_image_1.png',
 'Youtube_assault\\YT_optical_flow_image_2.png']

In [4]:
import cv2
import numpy as np

# Function to save the optical flow visualization in RGB and HSV
def save_optical_flow(prvs_frame, hsv_image, frame_number):
    filename_base = f'Youtube_assault/YT_assault_6_frame_{frame_number}'
    cv2.imwrite(f'{filename_base}_fb.png', prvs_frame)
    cv2.imwrite(f'{filename_base}_hsv.png', hsv_image)

# Initialize video capture with the video file
cap = cv2.VideoCapture('Youtube_assault/YT_assault_6.mp4')

# Read the first frame
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

frame_number = 0
while ret:
    ret, frame2 = cap.read()
    if not ret:
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2', rgb)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:  # Esc key to stop
        break
    elif k == ord('s'):  # Save on pressing 's'
        save_optical_flow(frame2, rgb, frame_number)
    
    prvs = next
    frame_number += 1

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()
